In [1]:
import rioxarray
import numpy as np
import os
from glob import glob
from tqdm.notebook import tqdm

In [2]:
images = sorted(glob("../data/Sample1_tile/image/*.tif"))
masks = sorted(glob("../data/Sample1_tile/mask/*.tif"))

In [3]:
X = []
Y = []
# samples = np.inf
samples = 500
count = 0
for image, mask in tqdm(list(zip(images, masks))):
    _,image_name = os.path.split(image)
    _,mask_name = os.path.split(mask)
    image_name,_ = os.path.splitext(image_name)
    mask_name,_ = os.path.splitext(mask_name)
    image_name = image_name.split('_')
    mask_name = mask_name.split('_')
    if(image_name[1] != mask_name[1] and image_name[2] != mask_name[2]):
        raise NameError(f"{image_name=} {mask_name=}")
    
    x = rioxarray.open_rasterio(image)
    y = rioxarray.open_rasterio(mask)

    x = x.data
    if(x.shape[1] < 512):
        pad = 512 - x.shape[1]
        pad = np.zeros(shape=(3,pad,x.shape[2]))
        x = np.concatenate([x,pad], axis=1)
    if(x.shape[2] < 512):
        pad = 512 - x.shape[2]
        pad = np.zeros(shape=(3,x.shape[1],pad))
        x = np.concatenate([x,pad], axis=2)
    
    y = y.data
    if(y.shape[1] < 512):
        pad = 512 - y.shape[1]
        pad = np.zeros(shape=(1,pad,y.shape[2]))
        y = np.concatenate([y,pad], axis=1)
    if(y.shape[2] < 512):
        pad = 512 - y.shape[2]
        pad = np.zeros(shape=(1,y.shape[1],pad))
        y = np.concatenate([y,pad], axis=2)

    X.append(np.expand_dims(x, axis=0))
    Y.append(y)
    count += 1
    if(count >= samples):
        break


  0%|          | 0/21509 [00:00<?, ?it/s]

In [4]:
X = np.vstack(X).astype(np.float32)
Y = np.vstack(Y).astype(np.float32)
X.shape, Y.shape

((500, 3, 512, 512), (500, 512, 512))

In [5]:
X = X.swapaxes(1,2).swapaxes(2,3)

In [6]:
X.shape

(500, 512, 512, 3)

In [7]:
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm

2024-07-22 10:44:36.143747: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 10:44:36.155142: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 10:44:36.158578: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-22 10:44:36.167416: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-22 10:44:38.426457: W tensorflow/compiler/tf2

Segmentation Models: using `tf.keras` framework.


In [8]:
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
import keras

BACKBONE = 'inceptionresnetv2'
preprocess_input = get_preprocessing(BACKBONE)

# load your data
# x_train, y_train, x_val, y_val = load_data(...)

# preprocess input
X = preprocess_input(X)
# define model
model = Unet(BACKBONE, classes=1, encoder_weights='imagenet', encoder_freeze=True,)
loss = sm.losses.DiceLoss()
metrics = [sm.metrics.IOUScore(),'binary_accuracy',keras.metrics.Precision(),keras.metrics.Recall()]
model.compile("Adam", loss=loss, metrics=metrics)

# display_cb = DisplayCallback()
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights='True')
checkpoint_cb = keras.callbacks.ModelCheckpoint('../weights/model_unet-inceptionresnetv2_cp.keras', verbose=1, save_best_only=True)

# fit model
model.fit(
    x=X,
    y=Y,
    batch_size=1,
    epochs=100,
    validation_data=(X, Y),
    callbacks=[early_stopping_cb, checkpoint_cb],
)

I0000 00:00:1721619880.485456  174227 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721619880.512080  174227 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721619880.512271  174227 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721619880.513883  174227 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Epoch 1/100


I0000 00:00:1721619918.028829  174587 service.cc:146] XLA service 0x7f7a28003840 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721619918.028861  174587 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2024-07-22 10:45:19.447761: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-22 10:45:24.029564: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-07-22 10:45:35.158046: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 9.16GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-07-22 10:45:35.369747: W external/local_tsl/tsl/framework/bfc_allocator.cc:291]

  1/500 ━━━━━━━━━━━━━━━━━━━━ 10:18:56 74s/step - binary_accuracy: 0.2730 - iou_score: 6.4052e-11 - loss: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00

I0000 00:00:1721619960.368326  174587 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - binary_accuracy: 0.5720 - iou_score: 6.7686e-04 - loss: 0.9984 - precision: 0.0014 - recall: 0.8070
Epoch 1: val_loss improved from inf to 0.99909, saving model to ../weights/model_unet-inceptionresnetv2_cp.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 125s 101ms/step - binary_accuracy: 0.5722 - iou_score: 6.7624e-04 - loss: 0.9984 - precision: 0.0014 - recall: 0.8070 - val_binary_accuracy: 0.6683 - val_iou_score: 4.1574e-04 - val_loss: 0.9991 - val_precision: 0.0012 - val_recall: 1.0000
Epoch 2/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - binary_accuracy: 0.7471 - iou_score: 3.8470e-04 - loss: 0.9986 - precision: 0.0012 - recall: 0.7049
Epoch 2: val_loss improved from 0.99909 to 0.99904, saving model to ../weights/model_unet-inceptionresnetv2_cp.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 41s 82ms/step - binary_accuracy: 0.7472 - iou_score: 3.8465e-04 - loss: 0.9986 - precision: 0.0012 - recall: 0.7051 - val_binary_accuracy: 0.6942 - val_iou_score: 4.2785e-0

KeyboardInterrupt: 